# Dataset Raw

In [3]:
import os
import pandas as pd
import librosa
import numpy as np
from tqdm import tqdm
from PIL import Image

# === CONFIGURACIÓN ===
CSV_PATH = "data/ESC-50-master/meta/esc50.csv"
AUDIO_DIR = "data/ESC-50-master/audio"
OUTPUT_DIR = "data/spectrograms1/base"
SR = 22050
IMG_SIZE = (224, 224)

# Divisiones basadas en folds
TRAIN_FOLDS = [1, 2, 3]
VAL_FOLDS   = [4]
TEST_FOLDS  = [5]

# Crear carpetas base
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(OUTPUT_DIR, split), exist_ok=True)

# Leer metadatos
df = pd.read_csv(CSV_PATH)

def wav_to_spectrogram(wav_path, save_path):
    try:
        y, sr = librosa.load(wav_path, sr=SR)
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        S_db = librosa.power_to_db(S, ref=np.max)

        S_norm = (S_db - S_db.min()) / (S_db.max() - S_db.min())
        S_img = (S_norm * 255).astype(np.uint8)

        img = Image.fromarray(S_img).resize(IMG_SIZE).convert("L")
        img.save(save_path)
    except Exception as e:
        print(f"⚠️ Error procesando {wav_path}: {e}")

def process_split(split_name, folds):
    df_split = df[df["fold"].isin(folds)]
    for _, row in tqdm(df_split.iterrows(), total=len(df_split), desc=f"Procesando {split_name}"):
        file_name = row["filename"]
        label = row["category"]

        # Crear carpeta por clase
        class_dir = os.path.join(OUTPUT_DIR, split_name, label)
        os.makedirs(class_dir, exist_ok=True)

        wav_path = os.path.join(AUDIO_DIR, file_name)
        save_path = os.path.join(class_dir, file_name.replace(".wav", ".png"))

        if not os.path.exists(save_path):
            wav_to_spectrogram(wav_path, save_path)

# Generar los tres splits
process_split("train", TRAIN_FOLDS)
process_split("val", VAL_FOLDS)
process_split("test", TEST_FOLDS)

print("✅ Espectrogramas generados y organizados por división.")


Procesando test: 100%|██████████| 400/400 [00:10<00:00, 36.91it/s]

✅ Espectrogramas generados y organizados por división.


# Modelo A

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet5(nn.Module):
    def __init__(self, num_classes=50, dropout=0.5):
        super(LeNet5, self).__init__()

        # --- Bloque 1 ---
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, padding=2)
        self.pool1 = nn.AvgPool2d(2, 2)

        # --- Bloque 2 ---
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.pool2 = nn.AvgPool2d(2, 2)

        # --- Bloque 3 ---
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3)
        self.pool3 = nn.AvgPool2d(2, 2)

        # --- Normalización por capa (importante para tanh) ---
        self.bn1 = nn.BatchNorm2d(6)
        self.bn2 = nn.BatchNorm2d(16)
        self.bn3 = nn.BatchNorm2d(32)

        # --- Capas densas ---
        self.fc1 = nn.Linear(32 * 26 * 26, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)
        self.dropout = nn.Dropout(p=dropout)

        # Inicialización recomendada para tanh
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        # Normalización ligera antes de tanh para evitar saturación
        x = self.pool1(torch.tanh(self.bn1(self.conv1(x))))
        x = self.pool2(torch.tanh(self.bn2(self.conv2(x))))
        x = self.pool3(torch.tanh(self.bn3(self.conv3(x))))
        x = x.view(x.size(0), -1)
        x = torch.tanh(self.fc1(x))
        x = self.dropout(x)
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x


## Entrenamiento Dataset Raw Modelo A

In [19]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Normalización [-1,1] porque LeNet usa tanh
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_data = datasets.ImageFolder("data/spectrograms1/base/train", transform=transform)
val_data   = datasets.ImageFolder("data/spectrograms1/base/val", transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32)


In [ ]:
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# --- Transformaciones ---
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_data = datasets.ImageFolder("data/spectrograms1_split/train", transform=transform)
val_data   = datasets.ImageFolder("data/spectrograms1_split/val", transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32)

# === LISTA DE EXPERIMENTOS ===
experiments = [
    {"optimizer": "Adam", "lr": 0.001, "batch_size": 32, "weight_decay": 0},
    {"optimizer": "Adam", "lr": 0.0005, "batch_size": 32, "weight_decay": 0.0001},
    {"optimizer": "Adam", "lr": 0.001, "batch_size": 64, "weight_decay": 0},
    {"optimizer": "SGD",  "lr": 0.01,  "batch_size": 32, "weight_decay": 0.0001},
    {"optimizer": "SGD",  "lr": 0.001, "batch_size": 16, "weight_decay": 0},
]

# === ENTRENAR 5 RUNS ===
for i, exp in enumerate(experiments, start=1):
    wandb.init(
        project="esc50-lenet",
        name=f"run_{i}_opt-{exp['optimizer']}_lr-{exp['lr']}_bs-{exp['batch_size']}",
        config=exp
    )
    config = wandb.config

    # Actualizar batch size dinámicamente
    train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
    val_loader   = DataLoader(val_data, batch_size=config.batch_size)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LeNet5(num_classes=len(train_data.classes)).to(device)

    criterion = nn.CrossEntropyLoss()

    if config.optimizer == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    else:
        optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=0.9, weight_decay=config.weight_decay)

    best_val_acc = 0.0
    EPOCHS = 40

    for epoch in range(EPOCHS):
        model.train()
        running_loss, correct, total = 0, 0, 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

        train_acc = correct / total
        train_loss = running_loss / len(train_loader)

        # Validación
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (preds == labels).sum().item()

        val_acc = val_correct / val_total
        val_loss /= len(val_loader)

        wandb.log({
            "epoch": epoch+1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        })

        print(f"[Run {i}] Epoch {epoch+1}/{EPOCHS} | Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"models/lenet_best_run{i}.pth")
            wandb.run.summary["best_val_acc"] = best_val_acc

    wandb.finish()


[Run 1] Epoch 1/40 | Train Acc: 0.046 | Val Acc: 0.110
[Run 1] Epoch 2/40 | Train Acc: 0.097 | Val Acc: 0.113
[Run 1] Epoch 3/40 | Train Acc: 0.130 | Val Acc: 0.143
[Run 1] Epoch 4/40 | Train Acc: 0.161 | Val Acc: 0.167
[Run 1] Epoch 5/40 | Train Acc: 0.189 | Val Acc: 0.203
[Run 1] Epoch 6/40 | Train Acc: 0.236 | Val Acc: 0.210
[Run 1] Epoch 7/40 | Train Acc: 0.271 | Val Acc: 0.223
[Run 1] Epoch 8/40 | Train Acc: 0.264 | Val Acc: 0.270
[Run 1] Epoch 9/40 | Train Acc: 0.305 | Val Acc: 0.260
[Run 1] Epoch 10/40 | Train Acc: 0.368 | Val Acc: 0.277
[Run 1] Epoch 11/40 | Train Acc: 0.386 | Val Acc: 0.267
[Run 1] Epoch 12/40 | Train Acc: 0.442 | Val Acc: 0.290
[Run 1] Epoch 13/40 | Train Acc: 0.473 | Val Acc: 0.300
[Run 1] Epoch 14/40 | Train Acc: 0.501 | Val Acc: 0.297
[Run 1] Epoch 15/40 | Train Acc: 0.546 | Val Acc: 0.283
[Run 1] Epoch 16/40 | Train Acc: 0.591 | Val Acc: 0.307
[Run 1] Epoch 17/40 | Train Acc: 0.634 | Val Acc: 0.290
[Run 1] Epoch 18/40 | Train Acc: 0.671 | Val Acc: 0.290
[

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇███████████
train_loss,██▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▂▃▄▄▄▆▅▆▆▆▆▆▆▇▆▆▇▇▇▇▇█▇█▆▇▇█▇█▇▇▇█▇▇▇▇
val_loss,█▇▆▅▄▄▃▃▂▂▂▁▂▁▁▁▁▁▁▂▂▂▂▁▂▂▃▂▃▃▃▃▃▃▄▄▄▄▅▄
best_val_acc,0.35333
epoch,40
train_acc,0.98
train_loss,0.17417
val_acc,0.31
val_loss,3.0484


[Run 2] Epoch 1/40 | Train Acc: 0.049 | Val Acc: 0.080
[Run 2] Epoch 2/40 | Train Acc: 0.096 | Val Acc: 0.080
[Run 2] Epoch 3/40 | Train Acc: 0.120 | Val Acc: 0.150
[Run 2] Epoch 4/40 | Train Acc: 0.160 | Val Acc: 0.187
[Run 2] Epoch 5/40 | Train Acc: 0.205 | Val Acc: 0.180
[Run 2] Epoch 6/40 | Train Acc: 0.254 | Val Acc: 0.237
[Run 2] Epoch 7/40 | Train Acc: 0.291 | Val Acc: 0.230
[Run 2] Epoch 8/40 | Train Acc: 0.340 | Val Acc: 0.267
[Run 2] Epoch 9/40 | Train Acc: 0.393 | Val Acc: 0.283
[Run 2] Epoch 10/40 | Train Acc: 0.458 | Val Acc: 0.273
[Run 2] Epoch 11/40 | Train Acc: 0.515 | Val Acc: 0.297
[Run 2] Epoch 12/40 | Train Acc: 0.551 | Val Acc: 0.300
[Run 2] Epoch 13/40 | Train Acc: 0.622 | Val Acc: 0.337
[Run 2] Epoch 14/40 | Train Acc: 0.655 | Val Acc: 0.297
[Run 2] Epoch 15/40 | Train Acc: 0.686 | Val Acc: 0.323
[Run 2] Epoch 16/40 | Train Acc: 0.732 | Val Acc: 0.323
[Run 2] Epoch 17/40 | Train Acc: 0.779 | Val Acc: 0.333
[Run 2] Epoch 18/40 | Train Acc: 0.802 | Val Acc: 0.323
[

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc,▁▁▂▂▂▃▃▃▄▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇███████████████
train_loss,██▇▇▇▇▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▃▄▄▅▅▆▆▆▇▇█▇▇▇█▇▇▇█▇▇▇▇▇▇██▇█▇██▇█▇▇▇▇
val_loss,█▇▆▅▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▂▃▃▃
best_val_acc,0.34333
epoch,40
train_acc,0.99143
train_loss,0.17701
val_acc,0.32
val_loss,2.81106


[Run 3] Epoch 1/40 | Train Acc: 0.049 | Val Acc: 0.103
[Run 3] Epoch 2/40 | Train Acc: 0.094 | Val Acc: 0.100
[Run 3] Epoch 3/40 | Train Acc: 0.126 | Val Acc: 0.150
[Run 3] Epoch 4/40 | Train Acc: 0.156 | Val Acc: 0.170
[Run 3] Epoch 5/40 | Train Acc: 0.187 | Val Acc: 0.197
[Run 3] Epoch 6/40 | Train Acc: 0.219 | Val Acc: 0.173
[Run 3] Epoch 7/40 | Train Acc: 0.228 | Val Acc: 0.190
[Run 3] Epoch 8/40 | Train Acc: 0.261 | Val Acc: 0.227
[Run 3] Epoch 9/40 | Train Acc: 0.311 | Val Acc: 0.280
[Run 3] Epoch 10/40 | Train Acc: 0.314 | Val Acc: 0.253
[Run 3] Epoch 11/40 | Train Acc: 0.371 | Val Acc: 0.267
[Run 3] Epoch 12/40 | Train Acc: 0.399 | Val Acc: 0.277
[Run 3] Epoch 13/40 | Train Acc: 0.472 | Val Acc: 0.297
[Run 3] Epoch 14/40 | Train Acc: 0.489 | Val Acc: 0.307
[Run 3] Epoch 15/40 | Train Acc: 0.543 | Val Acc: 0.280
[Run 3] Epoch 16/40 | Train Acc: 0.576 | Val Acc: 0.337
[Run 3] Epoch 17/40 | Train Acc: 0.621 | Val Acc: 0.303
[Run 3] Epoch 18/40 | Train Acc: 0.676 | Val Acc: 0.337
[

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc,▁▁▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇██████████
train_loss,██▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val_acc,▁▁▂▃▃▃▃▄▆▅▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇██▇▇█▇▇██▇█▇▇▇
val_loss,█▇▆▅▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▂▁▂▂▂▂▂
best_val_acc,0.37333
epoch,40
train_acc,0.95
train_loss,0.32957
val_acc,0.34333
val_loss,2.63842


[Run 4] Epoch 1/40 | Train Acc: 0.053 | Val Acc: 0.070
[Run 4] Epoch 2/40 | Train Acc: 0.114 | Val Acc: 0.123
[Run 4] Epoch 3/40 | Train Acc: 0.156 | Val Acc: 0.157
[Run 4] Epoch 4/40 | Train Acc: 0.195 | Val Acc: 0.187
[Run 4] Epoch 5/40 | Train Acc: 0.229 | Val Acc: 0.227
[Run 4] Epoch 6/40 | Train Acc: 0.238 | Val Acc: 0.270
[Run 4] Epoch 7/40 | Train Acc: 0.272 | Val Acc: 0.243
[Run 4] Epoch 8/40 | Train Acc: 0.284 | Val Acc: 0.257
[Run 4] Epoch 9/40 | Train Acc: 0.304 | Val Acc: 0.260
[Run 4] Epoch 10/40 | Train Acc: 0.344 | Val Acc: 0.280
[Run 4] Epoch 11/40 | Train Acc: 0.373 | Val Acc: 0.277
[Run 4] Epoch 12/40 | Train Acc: 0.372 | Val Acc: 0.297
[Run 4] Epoch 13/40 | Train Acc: 0.431 | Val Acc: 0.297
[Run 4] Epoch 14/40 | Train Acc: 0.484 | Val Acc: 0.300
[Run 4] Epoch 15/40 | Train Acc: 0.496 | Val Acc: 0.293
[Run 4] Epoch 16/40 | Train Acc: 0.571 | Val Acc: 0.330
[Run 4] Epoch 17/40 | Train Acc: 0.606 | Val Acc: 0.337
[Run 4] Epoch 18/40 | Train Acc: 0.664 | Val Acc: 0.330
[

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc,▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▆▆▆▆▇▇▇▇▇██████████████
train_loss,█▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▂▃▄▅▆▅▆▆▆▆▇▇▇▇▇█▇▇█▇██▇▇█▇█▇▇▇▇▇▇▇█████
val_loss,█▆▅▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▂▂▂▃▂▃▂▃▃▄▃▄▄▄▄▄▄▄▄▄▄
best_val_acc,0.35333
epoch,40
train_acc,0.99286
train_loss,0.09715
val_acc,0.34333
val_loss,2.93547


[Run 5] Epoch 1/40 | Train Acc: 0.039 | Val Acc: 0.067
[Run 5] Epoch 2/40 | Train Acc: 0.056 | Val Acc: 0.083
[Run 5] Epoch 3/40 | Train Acc: 0.083 | Val Acc: 0.090
[Run 5] Epoch 4/40 | Train Acc: 0.107 | Val Acc: 0.100
[Run 5] Epoch 5/40 | Train Acc: 0.130 | Val Acc: 0.147
[Run 5] Epoch 6/40 | Train Acc: 0.159 | Val Acc: 0.157
[Run 5] Epoch 7/40 | Train Acc: 0.185 | Val Acc: 0.180
[Run 5] Epoch 8/40 | Train Acc: 0.210 | Val Acc: 0.210
[Run 5] Epoch 9/40 | Train Acc: 0.216 | Val Acc: 0.217
[Run 5] Epoch 10/40 | Train Acc: 0.257 | Val Acc: 0.227
[Run 5] Epoch 11/40 | Train Acc: 0.256 | Val Acc: 0.223
[Run 5] Epoch 12/40 | Train Acc: 0.265 | Val Acc: 0.243
[Run 5] Epoch 13/40 | Train Acc: 0.266 | Val Acc: 0.270
[Run 5] Epoch 14/40 | Train Acc: 0.281 | Val Acc: 0.257
[Run 5] Epoch 15/40 | Train Acc: 0.302 | Val Acc: 0.247
[Run 5] Epoch 16/40 | Train Acc: 0.302 | Val Acc: 0.273
[Run 5] Epoch 17/40 | Train Acc: 0.323 | Val Acc: 0.277
[Run 5] Epoch 18/40 | Train Acc: 0.322 | Val Acc: 0.287
[

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc,▁▁▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇███
train_loss,██▇▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
val_acc,▁▁▂▂▃▃▄▅▅▅▅▆▆▆▆▆▆▇▆▇▇▇█▇▇▇▇▇██▇█▇▇██▇██▇
val_loss,█▇▇▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
best_val_acc,0.34
epoch,40
train_acc,0.55286
train_loss,1.79318
val_acc,0.31667
val_loss,2.46487


## Dataset Augmented

In [9]:
import os
import random
import numpy as np
from PIL import Image
from tqdm import tqdm

# === CONFIGURACIÓN ===
BASE_DIR = "data/spectrograms1/base"
AUG_DIR = "data/spectrograms1/augmented"

# Parámetros de SpecAugment (según Park et al., 2019)
FREQ_MASK_PARAM = 20      # ancho máximo de bandas de frecuencia a enmascarar
TIME_MASK_PARAM = 25      # ancho máximo de regiones de tiempo a enmascarar
NUM_FREQ_MASKS = 2        # número de máscaras de frecuencia
NUM_TIME_MASKS = 2        # número de máscaras de tiempo

# Crear carpetas base del dataset aumentado
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(AUG_DIR, split), exist_ok=True)


# === FUNCIÓN PRINCIPAL ===
def apply_specaugment(image_path, save_path):
    """Aplica SpecAugment a una imagen de espectrograma (grises)."""
    try:
        img = Image.open(image_path).convert("L")
        spec = np.array(img)

        # --- Frequency masking (vertical) ---
        for _ in range(NUM_FREQ_MASKS):
            f = random.randint(0, FREQ_MASK_PARAM)
            if f == 0:
                continue
            f0 = random.randint(0, spec.shape[0] - f)
            spec[f0:f0 + f, :] = 0  # borra bandas de frecuencia

        # --- Time masking (horizontal) ---
        for _ in range(NUM_TIME_MASKS):
            t = random.randint(0, TIME_MASK_PARAM)
            if t == 0:
                continue
            t0 = random.randint(0, spec.shape[1] - t)
            spec[:, t0:t0 + t] = 0  # borra regiones de tiempo

        aug_img = Image.fromarray(spec)
        aug_img.save(save_path)
    except Exception as e:
        print(f"⚠️ Error procesando {image_path}: {e}")


# === RECORRER TODAS LAS PARTICIONES ===
def process_split(split_name):
    base_split_path = os.path.join(BASE_DIR, split_name)
    aug_split_path = os.path.join(AUG_DIR, split_name)

    # recorrer las clases
    for class_name in os.listdir(base_split_path):
        class_base_path = os.path.join(base_split_path, class_name)
        class_aug_path = os.path.join(aug_split_path, class_name)
        os.makedirs(class_aug_path, exist_ok=True)

        # recorrer las imágenes
        images = [f for f in os.listdir(class_base_path) if f.endswith(".png")]

        for img_file in tqdm(images, desc=f"{split_name}/{class_name}", leave=False):
            src_path = os.path.join(class_base_path, img_file)
            dst_path = os.path.join(class_aug_path, img_file)

            # aplicar SpecAugment y guardar
            apply_specaugment(src_path, dst_path)


# === EJECUCIÓN ===
for split in ["train", "val", "test"]:
    print(f"\nProcesando {split.upper()}...")
    process_split(split)

print("\n✅ Dataset aumentado generado con SpecAugment.")



Procesando TRAIN...



Procesando VAL...



Procesando TEST...



✅ Dataset aumentado generado con SpecAugment.


In [12]:
import os
import torch
import torchaudio
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

# === CONFIGURACIÓN ===
CSV_PATH = "data/ESC-50-master/meta/esc50.csv"
AUDIO_DIR = "data/ESC-50-master/audio"
OUTPUT_DIR = "data/spectrograms1/augmented1"

SAMPLE_RATE = 22050
N_MELS = 128
IMG_SIZE = (224, 224)

# Folds por división
TRAIN_FOLDS = [1, 2, 3]
VAL_FOLDS   = [4]
TEST_FOLDS  = [5]

# === TRANSFORMACIONES ===
mel_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_mels=N_MELS
)

specaugment = torchaudio.transforms.SpecAugment(
    freq_mask_param=15,      # enmascaramiento de frecuencia
    time_mask_param=35,      # enmascaramiento de tiempo
    n_freq_masks=2,
    n_time_masks=2
)

# === FUNCIONES ===
def save_spec_image(tensor, save_path):
    """Convierte un tensor de espectrograma a imagen PNG."""
    tensor = tensor.squeeze().numpy()
    tensor_db = torchaudio.functional.amplitude_to_DB(
        torch.tensor(tensor), multiplier=10, amin=1e-10, db_multiplier=0
    ).numpy()
    tensor_norm = (tensor_db - tensor_db.min()) / (tensor_db.max() - tensor_db.min())
    img = (tensor_norm * 255).astype(np.uint8)
    Image.fromarray(img).resize(IMG_SIZE).convert("L").save(save_path)

def process_split(df, split_name):
    """Procesa un subconjunto del dataset aplicando SpecAugment."""
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Procesando {split_name}"):
        file_name = row["filename"]
        label = row["category"]

        class_dir = os.path.join(OUTPUT_DIR, split_name, label)
        os.makedirs(class_dir, exist_ok=True)

        wav_path = os.path.join(AUDIO_DIR, file_name)
        save_path = os.path.join(class_dir, file_name.replace(".wav", ".png"))

        try:
            # Cargar audio
            waveform, sr = torchaudio.load(wav_path)
            if sr != SAMPLE_RATE:
                waveform = torchaudio.functional.resample(waveform, sr, SAMPLE_RATE)

            # Crear mel-espectrograma
            mel_spec = mel_transform(waveform)

            # Aplicar SpecAugment
            augmented_spec = specaugment(mel_spec)

            # Guardar como imagen
            save_spec_image(augmented_spec, save_path)

        except Exception as e:
            print(f"⚠️ Error con {wav_path}: {e}")

# === MAIN ===
df = pd.read_csv(CSV_PATH)

splits = {
    "train": df[df["fold"].isin(TRAIN_FOLDS)],
    "val":   df[df["fold"].isin(VAL_FOLDS)],
    "test":  df[df["fold"].isin(TEST_FOLDS)]
}

for split_name, subset in splits.items():
    process_split(subset, split_name)

print("\n✅ Dataset aumentado con SpecAugment generado correctamente.")


OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed

## Entrenamiento Dataset Augmented Modelo A

In [31]:
DATA_DIR = "data/spectrograms1/augmented"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==============================
# TRANSFORMACIONES
# ==============================
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_data = datasets.ImageFolder(f"{DATA_DIR}/train", transform=transform)
val_data   = datasets.ImageFolder(f"{DATA_DIR}/val", transform=transform)

In [32]:
experiments = [
    {"optimizer": "Adam", "lr": 0.001, "batch_size": 32, "weight_decay": 1e-4},
    {"optimizer": "Adam", "lr": 0.0005, "batch_size": 32, "weight_decay": 1e-4},
    {"optimizer": "Adam", "lr": 0.001, "batch_size": 64, "weight_decay": 1e-4},
    {"optimizer": "SGD",  "lr": 0.01,  "batch_size": 32, "weight_decay": 1e-4},
    {"optimizer": "SGD",  "lr": 0.001, "batch_size": 16, "weight_decay": 1e-4},
]

# ==============================
# CICLO DE ENTRENAMIENTO (IGUAL AL DATASET 1)
# ==============================
for i, exp in enumerate(experiments, start=1):
    wandb.init(
        project="esc50-lenet-augmented",
        name=f"run_{i}_opt-{exp['optimizer']}_lr-{exp['lr']}_bs-{exp['batch_size']}",
        config=exp
    )
    config = wandb.config

    train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True)
    val_loader   = DataLoader(val_data, batch_size=config.batch_size)

    model = LeNet5(num_classes=len(train_data.classes)).to(device)
    criterion = nn.CrossEntropyLoss()

    if config.optimizer == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    else:
        optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=0.9, weight_decay=config.weight_decay)

    scheduler = StepLR(optimizer, step_size=8, gamma=0.7)

    EPOCHS = 40
    best_val_acc = 0.0
    patience = 6
    patience_counter = 0

    for epoch in range(EPOCHS):
        model.train()
        running_loss, correct, total = 0, 0, 0

        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

        train_acc = correct / total
        train_loss = running_loss / len(train_loader)

        # === VALIDACIÓN ===
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (preds == labels).sum().item()

        val_acc = val_correct / val_total
        val_loss /= len(val_loader)
        scheduler.step()

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": scheduler.get_last_lr()[0],
        })

        print(f"[Run {i}] Epoch {epoch+1}/{EPOCHS} | "
              f"Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f} | "
              f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # === Early stopping ===
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), f"models/lenet5_aug_best_run{i}.pth")
            wandb.run.summary["best_val_acc"] = best_val_acc
        else:
            patience_counter += 1
            if patience_counter > patience:
                print(f"[Run {i}] Early stopping triggered at epoch {epoch+1}.")
                break

    print(f"✅ [Run {i}] Mejor Val Acc: {best_val_acc:.3f}")
    wandb.finish()

[Run 1] Epoch 1/40 | Train Acc: 0.048 | Val Acc: 0.058 | Train Loss: 3.7720 | Val Loss: 3.6313
[Run 1] Epoch 2/40 | Train Acc: 0.072 | Val Acc: 0.090 | Train Loss: 3.5444 | Val Loss: 3.5044
[Run 1] Epoch 3/40 | Train Acc: 0.098 | Val Acc: 0.083 | Train Loss: 3.4440 | Val Loss: 3.4397
[Run 1] Epoch 4/40 | Train Acc: 0.132 | Val Acc: 0.120 | Train Loss: 3.3288 | Val Loss: 3.3846
[Run 1] Epoch 5/40 | Train Acc: 0.141 | Val Acc: 0.133 | Train Loss: 3.2640 | Val Loss: 3.3304
[Run 1] Epoch 6/40 | Train Acc: 0.138 | Val Acc: 0.140 | Train Loss: 3.2181 | Val Loss: 3.2802
[Run 1] Epoch 7/40 | Train Acc: 0.160 | Val Acc: 0.170 | Train Loss: 3.1562 | Val Loss: 3.2503
[Run 1] Epoch 8/40 | Train Acc: 0.172 | Val Acc: 0.140 | Train Loss: 3.1305 | Val Loss: 3.2696
[Run 1] Epoch 9/40 | Train Acc: 0.194 | Val Acc: 0.128 | Train Loss: 3.0511 | Val Loss: 3.2524
[Run 1] Epoch 10/40 | Train Acc: 0.204 | Val Acc: 0.130 | Train Loss: 2.9838 | Val Loss: 3.2430
[Run 1] Epoch 11/40 | Train Acc: 0.219 | Val Acc:

epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇██
lr,███████▄▄▄▄▄▄▄▄▁▁▁▁▁▁▁▁
train_acc,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▇▇▇█
train_loss,█▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▂▂▂▂▁▁
val_acc,▁▃▂▄▅▅▆▅▄▄▅▅▇▆▇█▆▆▇▇██▇
val_loss,█▆▅▅▄▃▃▃▃▃▃▃▂▁▁▁▂▂▁▁▁▁▁
best_val_acc,0.2025
epoch,23
lr,0.00049
train_acc,0.52083
train_loss,2.03415


[Run 2] Epoch 1/40 | Train Acc: 0.043 | Val Acc: 0.052 | Train Loss: 3.8529 | Val Loss: 3.7090
[Run 2] Epoch 2/40 | Train Acc: 0.082 | Val Acc: 0.068 | Train Loss: 3.5996 | Val Loss: 3.5511
[Run 2] Epoch 3/40 | Train Acc: 0.147 | Val Acc: 0.110 | Train Loss: 3.4077 | Val Loss: 3.4791
[Run 2] Epoch 4/40 | Train Acc: 0.174 | Val Acc: 0.125 | Train Loss: 3.2858 | Val Loss: 3.4160
[Run 2] Epoch 5/40 | Train Acc: 0.231 | Val Acc: 0.163 | Train Loss: 3.1617 | Val Loss: 3.3699
[Run 2] Epoch 6/40 | Train Acc: 0.286 | Val Acc: 0.145 | Train Loss: 3.0103 | Val Loss: 3.3281
[Run 2] Epoch 7/40 | Train Acc: 0.345 | Val Acc: 0.138 | Train Loss: 2.8824 | Val Loss: 3.3323
[Run 2] Epoch 8/40 | Train Acc: 0.368 | Val Acc: 0.170 | Train Loss: 2.7625 | Val Loss: 3.2698
[Run 2] Epoch 9/40 | Train Acc: 0.438 | Val Acc: 0.170 | Train Loss: 2.5974 | Val Loss: 3.2337
[Run 2] Epoch 10/40 | Train Acc: 0.487 | Val Acc: 0.203 | Train Loss: 2.4682 | Val Loss: 3.2199
[Run 2] Epoch 11/40 | Train Acc: 0.544 | Val Acc:

epoch,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
lr,███████▄▄▄▄▄▄▄▄▁▁
train_acc,▁▁▂▂▃▄▄▄▅▆▆▇▇▇▇██
train_loss,█▇▇▆▆▅▅▄▄▃▃▃▂▂▂▁▁
val_acc,▁▂▄▄▆▅▅▆▆█▇█▇▇███
val_loss,█▆▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁
best_val_acc,0.2025
epoch,17
lr,0.00024
train_acc,0.71417
train_loss,1.71079


[Run 3] Epoch 1/40 | Train Acc: 0.043 | Val Acc: 0.058 | Train Loss: 3.8034 | Val Loss: 3.6058
[Run 3] Epoch 2/40 | Train Acc: 0.091 | Val Acc: 0.092 | Train Loss: 3.5447 | Val Loss: 3.4712
[Run 3] Epoch 3/40 | Train Acc: 0.136 | Val Acc: 0.102 | Train Loss: 3.4030 | Val Loss: 3.3887
[Run 3] Epoch 4/40 | Train Acc: 0.169 | Val Acc: 0.135 | Train Loss: 3.2755 | Val Loss: 3.3231
[Run 3] Epoch 5/40 | Train Acc: 0.218 | Val Acc: 0.133 | Train Loss: 3.1499 | Val Loss: 3.2539
[Run 3] Epoch 6/40 | Train Acc: 0.241 | Val Acc: 0.147 | Train Loss: 3.0370 | Val Loss: 3.2316
[Run 3] Epoch 7/40 | Train Acc: 0.282 | Val Acc: 0.158 | Train Loss: 2.8991 | Val Loss: 3.1809
[Run 3] Epoch 8/40 | Train Acc: 0.324 | Val Acc: 0.165 | Train Loss: 2.7669 | Val Loss: 3.1441
[Run 3] Epoch 9/40 | Train Acc: 0.366 | Val Acc: 0.163 | Train Loss: 2.6337 | Val Loss: 3.1664
[Run 3] Epoch 10/40 | Train Acc: 0.425 | Val Acc: 0.172 | Train Loss: 2.4882 | Val Loss: 3.1387
[Run 3] Epoch 11/40 | Train Acc: 0.478 | Val Acc:

epoch,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
lr,███████▄▄▄▄▄▄▄▄▁▁
train_acc,▁▂▂▂▃▃▄▄▄▅▆▆▆▇▇██
train_loss,█▇▇▆▆▅▅▅▄▄▃▃▂▂▂▁▁
val_acc,▁▃▄▆▆▆▇█▇██▇██▇██
val_loss,█▆▅▄▃▃▂▂▂▁▁▁▁▁▁▂▁
best_val_acc,0.1725
epoch,17
lr,0.00049
train_acc,0.70167
train_loss,1.67584


[Run 4] Epoch 1/40 | Train Acc: 0.030 | Val Acc: 0.055 | Train Loss: 3.8714 | Val Loss: 3.7322
[Run 4] Epoch 2/40 | Train Acc: 0.069 | Val Acc: 0.107 | Train Loss: 3.6054 | Val Loss: 3.4959
[Run 4] Epoch 3/40 | Train Acc: 0.114 | Val Acc: 0.138 | Train Loss: 3.4025 | Val Loss: 3.4217
[Run 4] Epoch 4/40 | Train Acc: 0.163 | Val Acc: 0.147 | Train Loss: 3.2395 | Val Loss: 3.3113
[Run 4] Epoch 5/40 | Train Acc: 0.180 | Val Acc: 0.138 | Train Loss: 3.1176 | Val Loss: 3.2709
[Run 4] Epoch 6/40 | Train Acc: 0.217 | Val Acc: 0.125 | Train Loss: 2.9857 | Val Loss: 3.3042
[Run 4] Epoch 7/40 | Train Acc: 0.239 | Val Acc: 0.175 | Train Loss: 2.9135 | Val Loss: 3.1644
[Run 4] Epoch 8/40 | Train Acc: 0.273 | Val Acc: 0.177 | Train Loss: 2.7594 | Val Loss: 3.1090
[Run 4] Epoch 9/40 | Train Acc: 0.338 | Val Acc: 0.168 | Train Loss: 2.5928 | Val Loss: 3.1228
[Run 4] Epoch 10/40 | Train Acc: 0.374 | Val Acc: 0.190 | Train Loss: 2.4608 | Val Loss: 3.0803
[Run 4] Epoch 11/40 | Train Acc: 0.407 | Val Acc:

epoch,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██
lr,███████▅▅▅▅▅▅▅▅▃▃▃▃▃▃▃▃▁
train_acc,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇████
train_loss,█▇▇▇▆▆▆▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁▁▁
val_acc,▁▃▅▅▅▄▇▇▆▇█▆▇▇▇▇█▇▇████▇
val_loss,█▆▅▄▃▄▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▂▂▂
best_val_acc,0.2025
epoch,24
lr,0.00343
train_acc,0.87083
train_loss,0.81957


[Run 5] Epoch 1/40 | Train Acc: 0.030 | Val Acc: 0.033 | Train Loss: 3.8964 | Val Loss: 3.8509
[Run 5] Epoch 2/40 | Train Acc: 0.052 | Val Acc: 0.065 | Train Loss: 3.7887 | Val Loss: 3.7278
[Run 5] Epoch 3/40 | Train Acc: 0.077 | Val Acc: 0.070 | Train Loss: 3.6700 | Val Loss: 3.6275
[Run 5] Epoch 4/40 | Train Acc: 0.102 | Val Acc: 0.098 | Train Loss: 3.5662 | Val Loss: 3.5627
[Run 5] Epoch 5/40 | Train Acc: 0.113 | Val Acc: 0.102 | Train Loss: 3.4968 | Val Loss: 3.5199
[Run 5] Epoch 6/40 | Train Acc: 0.136 | Val Acc: 0.100 | Train Loss: 3.4331 | Val Loss: 3.4835
[Run 5] Epoch 7/40 | Train Acc: 0.162 | Val Acc: 0.128 | Train Loss: 3.3715 | Val Loss: 3.4573
[Run 5] Epoch 8/40 | Train Acc: 0.168 | Val Acc: 0.135 | Train Loss: 3.3202 | Val Loss: 3.4273
[Run 5] Epoch 9/40 | Train Acc: 0.192 | Val Acc: 0.140 | Train Loss: 3.2668 | Val Loss: 3.4107
[Run 5] Epoch 10/40 | Train Acc: 0.212 | Val Acc: 0.145 | Train Loss: 3.2205 | Val Loss: 3.3904
[Run 5] Epoch 11/40 | Train Acc: 0.199 | Val Acc:

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
lr,███████▅▅▅▅▅▅▅▅▃▃▃▃▃▃▃▃▁▁
train_acc,▁▁▂▃▃▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇████
train_loss,█▇▇▆▆▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
val_acc,▁▃▃▄▅▄▆▆▆▇▇▇█▇███████▇▇██
val_loss,█▇▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁
best_val_acc,0.17
epoch,25
lr,0.00034
train_acc,0.34417
train_loss,2.76239


# Modelo B

In [1]:
from typing import Callable, Optional, Type, List
import torch
import torch.nn as nn


# -------------------------
# Utilidades de convolución
# -------------------------
def conv3x3(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """Conv 3×3 con padding=1, sin bias (BN lo compensa)."""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """Conv 1×1 para proyección en atajos (ajustar canales/stride)."""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


# -------------------------
# Bloque residual "básico"
# -------------------------
class BasicBlock(nn.Module):
    """
    Estructura:
        Conv3x3 → BN → ReLU → Conv3x3 → BN → (Suma con atajo) → ReLU
    Donde el atajo (identity) puede incluir una proyección 1×1 si cambia
    la resolución (stride > 1) o el número de canales.
    """
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1   = norm_layer(planes)
        self.relu  = nn.ReLU(inplace=True)

        self.conv2 = conv3x3(planes, planes)
        self.bn2   = norm_layer(planes)

        self.downsample = downsample  # Proyección para el atajo, si aplica

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x  # Atajo

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # Alinear dimensiones del atajo si cambió stride o # de canales
        if self.downsample is not None:
            identity = self.downsample(x)

        out = out + identity
        out = self.relu(out)
        return out


# -----------
# ResNet base
# -----------
class ResNet(nn.Module):
    """
    Constructor general de ResNet con BasicBlock y configuración [2,2,2,2].
    Parámetros clave:
        - small_input=True: conv1=3×3 s=1 y sin MaxPool (mejor para 64–224 px).
        - small_input=False: conv1=7×7 s=2 + MaxPool (clásico de ResNet).
    """
    def __init__(
        self,
        block: Type[BasicBlock],
        layers: List[int],
        num_classes: int = 50,
        in_channels: int = 1,
        small_input: bool = True,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64

        # Capa inicial: variante "small_input" recomendada para espectrogramas
        if small_input:
            # Preserva más detalle inicial (sin MaxPool temprano)
            self.conv1   = nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1, bias=False)
            self.maxpool = nn.Identity()
        else:
            # Estilo ResNet clásico para entradas grandes
            self.conv1   = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
            self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.bn1  = norm_layer(64)
        self.relu = nn.ReLU(inplace=True)

        # Stages: [64, 128, 256, 512] con [2, 2, 2, 2] bloques
        self.layer1 = self._make_layer(block,  64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        # Cabeza de clasificación
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Global Average Pooling
        self.fc      = nn.Linear(512 * block.expansion, num_classes)

        # Inicialización recomendada para ReLU/BN
        self._init_weights()

    def _make_layer(self, block: Type[BasicBlock], planes: int, blocks: int, stride: int = 1) -> nn.Sequential:
        """
        Crea un stage con 'blocks' bloques. El primer bloque puede hacer downsample
        (stride=2) para reducir resolución y duplicar canales.
        """
        norm_layer = self._norm_layer
        downsample = None

        # Si cambia resolución o # de canales, proyectamos el atajo (1×1)
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _init_weights(self) -> None:
        """Inicialización Kaiming para conv; constantes para BN; normal para FC."""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1.0)
                nn.init.constant_(m.bias, 0.0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0.0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Entrada → conv1 → BN → ReLU → (posible MaxPool/Identity)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # Stages residuales
        x = self.layer1(x)  # 64
        x = self.layer2(x)  # 128
        x = self.layer3(x)  # 256
        x = self.layer4(x)  # 512

        # Cabeza
        x = self.avgpool(x)           # (B, 512, 1, 1)
        x = torch.flatten(x, 1)       # (B, 512)
        x = self.fc(x)                # (B, num_classes)
        return x


# -------------------------
# Fábrica de ResNet-18
# -------------------------
def resnet18_audio(num_classes: int = 50, in_channels: int = 1, small_input: bool = True) -> ResNet:
    """
    Retorna una ResNet-18 lista para espectrogramas:
        - num_classes: # de clases del dataset (ESC-50 → 50)
        - in_channels: 1 para grises; 3 si usas RGB (replicar canal)
        - small_input: True recomendado para ~128–224 px
    """
    return ResNet(
        block=BasicBlock,
        layers=[2, 2, 2, 2],
        num_classes=num_classes,
        in_channels=in_channels,
        small_input=small_input,
    )

## Entrenamiento Dataset Raw Modelo B

In [ ]:
# ===========================================
# ENTRENAMIENTO - MODELO B (ResNet-18 Audio)
# Dataset: data/spectrograms1/base (RAW, sin augment)
# Imagen: 228x228
# GPU: <= 4 GB compatible
# ===========================================

import os, random, gc
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import wandb

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from torch.amp import autocast, GradScaler
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

# -----------------------------
# 0) Configuración y utilidades
# -----------------------------
os.makedirs("models", exist_ok=True)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# -----------------------------
# 1) Data (RAW)
# -----------------------------
DATA_DIR = "data/spectrograms1/base"
IMG_SIZE = (228, 228)

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # [-1,1]
])

train_data = datasets.ImageFolder(f"{DATA_DIR}/train", transform=transform)
val_data   = datasets.ImageFolder(f"{DATA_DIR}/val",   transform=transform)

num_classes = len(train_data.classes)
class_names = train_data.classes
print("Clases detectadas:", num_classes)

# -----------------------------
# 2) Experimentos
# -----------------------------
experiments = [
    {"optimizer": "AdamW", "lr": 3e-4,  "batch_size": 8, "weight_decay": 1e-4},
    {"optimizer": "AdamW", "lr": 1e-4,  "batch_size": 8, "weight_decay": 1e-4},
    {"optimizer": "SGD",   "lr": 0.01,  "batch_size": 8, "weight_decay": 1e-4},
    {"optimizer": "SGD",   "lr": 0.005, "batch_size": 8, "weight_decay": 1e-4},
    {"optimizer": "AdamW", "lr": 5e-4,  "batch_size": 12, "weight_decay": 1e-4},
]

EPOCHS   = 40
PATIENCE = 6

# -----------------------------
# 3) Loop multi-run
# -----------------------------
for i, exp in enumerate(experiments, start=1):
    print(f"\n===== Iniciando experimento {i} =====")

    wandb.init(
        project="esc50-modelB",
        name=f"resnet18B_run_{i}_opt-{exp['optimizer']}_lr-{exp['lr']}_bs-{exp['batch_size']}",
        config=exp,
        # mode="offline",
    )
    config = wandb.config

    train_loader = DataLoader(
        train_data,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_data,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    # -------------------------------
    # Modelo (ResNet-18 personalizada)
    # -------------------------------
    model = resnet18_audio(num_classes=num_classes, in_channels=1, small_input=True).to(device)

    criterion = nn.CrossEntropyLoss()
    if config.optimizer == "AdamW":
        optimizer = optim.AdamW(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    else:
        optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=0.9, weight_decay=config.weight_decay)

    scheduler = StepLR(optimizer, step_size=8, gamma=0.7)
    scaler = GradScaler('cuda' if torch.cuda.is_available() else 'cpu')

    best_val_acc = 0.0
    patience_counter = 0

    # -------------------------------
    # Entrenamiento por épocas
    # -------------------------------
    for epoch in range(EPOCHS):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for imgs, labels in train_loader:
            imgs = imgs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)

            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                outputs = model(imgs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item()
            total += labels.size(0)
            correct += (preds == labels).sum().item()

            # Limpieza batch
            del imgs, labels, outputs, loss, preds
            torch.cuda.empty_cache()

        train_acc = correct / total
        train_loss = running_loss / max(1, len(train_loader))

        # --------- Validación ---------
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        val_y_true, val_y_pred = [], []

        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs = imgs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                    outputs = model(imgs)
                    loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)
                val_loss += loss.item()
                val_total += labels.size(0)
                val_correct += (preds == labels).sum().item()

                val_y_true.extend(labels.cpu().tolist())
                val_y_pred.extend(preds.cpu().tolist())

                del imgs, labels, outputs, preds, loss
                torch.cuda.empty_cache()

        val_acc = val_correct / val_total
        val_loss = val_loss / max(1, len(val_loader))

        # --- Métricas adicionales
        prec_m, rec_m, f1_m, _ = precision_recall_fscore_support(
            val_y_true, val_y_pred, average="macro", zero_division=0
        )
        prec_w, rec_w, f1_w, _ = precision_recall_fscore_support(
            val_y_true, val_y_pred, average="weighted", zero_division=0
        )

        scheduler.step()
        gc.collect()

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "val_f1_macro": f1_m,
            "val_f1_weighted": f1_w,
            "val_precision_macro": prec_m,
            "val_recall_macro": rec_m,
            "lr": scheduler.get_last_lr()[0],
            "val_confusion_matrix": wandb.plot.confusion_matrix(
                y_true=val_y_true,
                preds=val_y_pred,
                class_names=class_names
            )
        })

        print(f"[Run {i}] Ep {epoch+1:02d}/{EPOCHS} | "
              f"Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f} | "
              f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # --- Early stopping y guardado
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), f"models/MODEL_B_resnet18_audio_best_run{i}.pth")
            wandb.run.summary["best_val_acc"] = best_val_acc
        else:
            patience_counter += 1
            if patience_counter > PATIENCE:
                print(f"[Run {i}] Early stopping en epoch {epoch+1}.")
                break

        torch.cuda.empty_cache()
        gc.collect()

    print(f"✅ [Run {i}] Mejor Val Acc: {best_val_acc:.3f}")

    wandb.finish()
    del model, optimizer, scheduler, scaler
    torch.cuda.empty_cache()
    gc.collect()


Device: cuda
Clases detectadas: 50

===== Iniciando experimento 1 =====


/tmp/ipykernel_15696/528800939.py:114: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_15696/528800939.py:132: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_15696/528800939.py:162: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


[Run 1] Ep 01/40 | Train Acc: 0.074 | Val Acc: 0.083 | Train Loss: 3.5868 | Val Loss: 3.5775


/tmp/ipykernel_15696/528800939.py:132: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/tmp/ipykernel_15696/528800939.py:162: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


[Run 1] Ep 02/40 | Train Acc: 0.147 | Val Acc: 0.145 | Train Loss: 3.0707 | Val Loss: 3.1812


/tmp/ipykernel_15696/528800939.py:132: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x78ff85296330>> (for post_run_cell), with arguments args (<ExecutionResult object at 790032def650, execution_count=3 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 790032dedfd0, raw_cell="# ===========================================
# EN.." transformed_cell="# ===========================================
# EN.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/javialroro/TEC/2025%20II/IA/CNNVoiceRecognition/source.ipynb#X22sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: Connection lost

## Entrenamiento Dataset Augmented Modelo B

In [2]:
# ===========================================
# ENTRENAMIENTO - MODELO B (ResNet-18 Audio)
# Dataset: data/spectrograms1/augmented
# Optimizado para GPUs pequeñas (≤ 4 GB)
# ===========================================

import os, random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import wandb

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from torch.amp import autocast, GradScaler
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

# ---- 0) Setup
os.makedirs("models", exist_ok=True)
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---- 1) Data
DATA_DIR = "data/spectrograms1/augmented"
IMG_SIZE = (224, 224)   

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_data = datasets.ImageFolder(f"{DATA_DIR}/train", transform=transform)
val_data   = datasets.ImageFolder(f"{DATA_DIR}/val",   transform=transform)

num_classes = len(train_data.classes)
class_names = train_data.classes
print("Clases:", num_classes)

# ---- 2) Experimentos
experiments = [
    {"optimizer": "Adam", "lr": 0.001,  "batch_size": 8,  "weight_decay": 1e-4},
    {"optimizer": "Adam", "lr": 0.0005, "batch_size": 8,  "weight_decay": 1e-4},
    {"optimizer": "Adam", "lr": 0.001,  "batch_size": 16, "weight_decay": 1e-4},
    {"optimizer": "SGD",  "lr": 0.01,   "batch_size": 8,  "weight_decay": 1e-4},
    {"optimizer": "SGD",  "lr": 0.001,  "batch_size": 8,  "weight_decay": 1e-4},
]

EPOCHS   = 40
PATIENCE = 6

# ---- 3) Loop multi-run
for i, exp in enumerate(experiments, start=1):
    # Cerrar cualquier run previo de W&B para evitar duplicados
    if wandb.run is not None:
        wandb.finish()
    
    wandb.init(
        project="esc50-resnet18-augmented",  # ← Nombre único del proyecto
        name=f"run_{i}_opt-{exp['optimizer']}_lr-{exp['lr']}_bs-{exp['batch_size']}",
        config=exp,
        reinit=True  # Permite reinicialización si hay conflictos
    )
    config = wandb.config

    train_loader = DataLoader(
        train_data, batch_size=config.batch_size, shuffle=True, num_workers=2, pin_memory=True
    )
    val_loader   = DataLoader(
        val_data,   batch_size=config.batch_size, shuffle=False, num_workers=2, pin_memory=True
    )

    # --- Modelo B (tu ResNet-18)
    model = resnet18_audio(num_classes=num_classes, in_channels=1, small_input=True).to(device)

    # --- Criterio / Optimizador / Scheduler
    criterion = nn.CrossEntropyLoss()
    if config.optimizer == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=config.lr, weight_decay=config.weight_decay)
    else:
        optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=0.9, weight_decay=config.weight_decay)

    scheduler = StepLR(optimizer, step_size=8, gamma=0.7)
    scaler = GradScaler('cuda' if torch.cuda.is_available() else 'cpu')

    best_val_acc = 0.0
    patience_counter = 0

    # ---- Entrenamiento
    for epoch in range(EPOCHS):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                outputs = model(imgs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

        train_acc = correct / total
        train_loss = running_loss / max(1, len(train_loader))

        # ---- Validación
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        val_y_true, val_y_pred = [], []
        
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                    outputs = model(imgs)
                    loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (preds == labels).sum().item()
                
                val_y_true.extend(labels.cpu().tolist())
                val_y_pred.extend(preds.cpu().tolist())

        val_acc = val_correct / val_total
        val_loss = val_loss / max(1, len(val_loader))

        # --- Métricas adicionales
        prec_m, rec_m, f1_m, _ = precision_recall_fscore_support(
            val_y_true, val_y_pred, average="macro", zero_division=0
        )
        prec_w, rec_w, f1_w, _ = precision_recall_fscore_support(
            val_y_true, val_y_pred, average="weighted", zero_division=0
        )

        scheduler.step()
        torch.cuda.empty_cache()  # Liberar VRAM

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "val_f1_macro": f1_m,
            "val_f1_weighted": f1_w,
            "val_precision_macro": prec_m,
            "val_recall_macro": rec_m,
            "lr": scheduler.get_last_lr()[0],
            "val_confusion_matrix": wandb.plot.confusion_matrix(
                y_true=val_y_true,
                preds=val_y_pred,
                class_names=class_names
            )
        })

        print(f"[Run {i}] Epoch {epoch+1:02d}/{EPOCHS} | "
              f"Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f} | "
              f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # ---- Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), f"models/resnet18_audio_AUG_best_run{i}.pth")
            wandb.run.summary["best_val_acc"] = best_val_acc
        else:
            patience_counter += 1
            if patience_counter > PATIENCE:
                print(f"[Run {i}] Early stopping triggered at epoch {epoch+1}.")
                break

    print(f"✅ [Run {i}] Mejor Val Acc: {best_val_acc:.3f}")
    wandb.finish()


/home/javialroro/miniconda3/envs/ml/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/javialroro/miniconda3/envs/ml/lib/python3.13/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened b

Device: cuda
Clases: 50


wandb: Currently logged in as: javialroro (javialroro-tecnologico-de-costa-rica) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[Run 1] Epoch 01/40 | Train Acc: 0.028 | Val Acc: 0.062 | Train Loss: 3.9371 | Val Loss: 3.7019
[Run 1] Epoch 02/40 | Train Acc: 0.052 | Val Acc: 0.065 | Train Loss: 3.6587 | Val Loss: 3.5678
[Run 1] Epoch 02/40 | Train Acc: 0.052 | Val Acc: 0.065 | Train Loss: 3.6587 | Val Loss: 3.5678
[Run 1] Epoch 03/40 | Train Acc: 0.059 | Val Acc: 0.095 | Train Loss: 3.5390 | Val Loss: 3.6986
[Run 1] Epoch 03/40 | Train Acc: 0.059 | Val Acc: 0.095 | Train Loss: 3.5390 | Val Loss: 3.6986
[Run 1] Epoch 04/40 | Train Acc: 0.114 | Val Acc: 0.125 | Train Loss: 3.2836 | Val Loss: 3.1744
[Run 1] Epoch 04/40 | Train Acc: 0.114 | Val Acc: 0.125 | Train Loss: 3.2836 | Val Loss: 3.1744
[Run 1] Epoch 05/40 | Train Acc: 0.128 | Val Acc: 0.188 | Train Loss: 3.1629 | Val Loss: 2.9689
[Run 1] Epoch 05/40 | Train Acc: 0.128 | Val Acc: 0.188 | Train Loss: 3.1629 | Val Loss: 2.9689
[Run 1] Epoch 06/40 | Train Acc: 0.149 | Val Acc: 0.087 | Train Loss: 2.9977 | Val Loss: 4.6198
[Run 1] Epoch 06/40 | Train Acc: 0.149 |

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,███████▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁
train_acc,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇██████
train_loss,█▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_acc,▁▁▁▂▃▁▃▃▄▄▄▄▄▅▃▅▃▅▆▅▆▆▆▆▇▇▇▇▇▇▇▆▇██▇█▇▇█
val_f1_macro,▁▁▁▂▂▁▃▃▄▄▄▄▄▅▃▅▃▅▆▅▆▆▆▆▇▇▇▇▇▇▇▆▇██▇█▇▇▇
val_f1_weighted,▁▁▁▂▂▁▃▃▄▄▄▄▄▅▃▅▃▅▆▅▆▆▆▆▇▇▇▇▇▇▇▆▇██▇█▇▇▇
val_loss,▆▆▆▅▄█▄▄▄▃▃▃▃▃▆▄▄▃▂▃▂▂▂▂▂▂▁▁▁▂▂▂▁▁▁▂▁▁▁▁
val_precision_macro,▁▁▁▂▂▂▃▂▄▄▅▄▄▅▄▅▄▆▆▆▆▆▆▇▇▇▇▇██▇▇▇▇█▇█▇██
val_recall_macro,▁▁▁▂▃▁▃▃▄▄▄▄▄▅▃▅▃▅▆▅▆▆▆▆▇▇▇▇▇▇▇▆▇██▇█▇▇█
best_val_acc,0.6125


...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,███████▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁
train_acc,▁▁▂▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███████
train_loss,█▇▇▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_acc,▁▁▂▃▃▄▅▄▄▄▄▅▅▅▅▅▇▅▆▇▇▇▆▆▇▇▇█▆██▇█▇█████▇
val_f1_macro,▁▁▁▂▃▄▄▄▃▄▄▅▅▅▅▄▇▅▆▆▇▇▆▅▇▇▇█▆██▇█▇█████▇
val_f1_weighted,▁▁▁▂▃▄▄▄▃▄▄▅▅▅▅▄▇▅▆▆▇▇▆▅▇▇▇█▆██▇█▇█████▇
val_loss,█▆▇▅▅▄▄▄▅▅▅▃▃▄▃▄▂▄▃▂▂▂▃▃▂▁▂▁▃▁▁▂▁▂▂▁▁▁▂▂
val_precision_macro,▁▁▁▂▃▄▅▄▄▄▅▅▅▆▅▆▇▆▆▇▇▇▇▆▇▇▇▇▇██▇▇██▇███▇
val_recall_macro,▁▁▂▃▃▄▅▄▄▄▄▅▅▅▅▅▇▅▆▇▇▇▆▆▇▇▇█▆██▇█▇█████▇
best_val_acc,0.62


[Run 4] Epoch 01/40 | Train Acc: 0.022 | Val Acc: 0.030 | Train Loss: 4.0767 | Val Loss: 3.9781
[Run 4] Epoch 02/40 | Train Acc: 0.056 | Val Acc: 0.075 | Train Loss: 3.6328 | Val Loss: 4.0524
[Run 4] Epoch 02/40 | Train Acc: 0.056 | Val Acc: 0.075 | Train Loss: 3.6328 | Val Loss: 4.0524
[Run 4] Epoch 03/40 | Train Acc: 0.112 | Val Acc: 0.083 | Train Loss: 3.3671 | Val Loss: 4.7000
[Run 4] Epoch 03/40 | Train Acc: 0.112 | Val Acc: 0.083 | Train Loss: 3.3671 | Val Loss: 4.7000
[Run 4] Epoch 04/40 | Train Acc: 0.144 | Val Acc: 0.177 | Train Loss: 3.1378 | Val Loss: 2.8800
[Run 4] Epoch 04/40 | Train Acc: 0.144 | Val Acc: 0.177 | Train Loss: 3.1378 | Val Loss: 2.8800
[Run 4] Epoch 05/40 | Train Acc: 0.183 | Val Acc: 0.145 | Train Loss: 2.9245 | Val Loss: 3.5701
[Run 4] Epoch 05/40 | Train Acc: 0.183 | Val Acc: 0.145 | Train Loss: 2.9245 | Val Loss: 3.5701
[Run 4] Epoch 06/40 | Train Acc: 0.228 | Val Acc: 0.250 | Train Loss: 2.7629 | Val Loss: 2.7424
[Run 4] Epoch 06/40 | Train Acc: 0.228 |

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
lr,███████▅▅▅▅▅▅▅▅▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
train_acc,▁▁▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██████
train_loss,█▇▇▆▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_acc,▁▂▂▃▂▄▄▃▄▅▅▅▅▅▆▅▆▇▇▆▇▆▆▆█▇▇██▇▇▇███
val_f1_macro,▁▁▁▂▂▃▃▃▄▅▅▅▅▅▅▅▆▇▆▅▇▆▆▆█▇▇██▇▇▇███
val_f1_weighted,▁▁▁▂▂▃▃▃▄▅▅▅▅▅▅▅▆▇▆▅▇▆▆▆█▇▇██▇▇▇███
val_loss,▆▇█▄▆▄▄▄▄▃▂▃▃▂▃▂▂▂▂▃▂▂▂▃▁▁▁▁▁▂▁▂▁▁▁
val_precision_macro,▁▁▁▂▂▃▃▃▄▅▆▆▅▅▆▆▇▇▆▆▇▆▆▆█▇▇██▇▇▇██▇
val_recall_macro,▁▂▂▃▂▄▄▃▄▅▅▅▅▅▆▅▆▇▇▆▇▆▆▆█▇▇██▇▇▇███
best_val_acc,0.6325


[Run 5] Epoch 01/40 | Train Acc: 0.025 | Val Acc: 0.060 | Train Loss: 3.9136 | Val Loss: 3.7168
[Run 5] Epoch 02/40 | Train Acc: 0.042 | Val Acc: 0.055 | Train Loss: 3.6964 | Val Loss: 3.5477
[Run 5] Epoch 02/40 | Train Acc: 0.042 | Val Acc: 0.055 | Train Loss: 3.6964 | Val Loss: 3.5477
[Run 5] Epoch 03/40 | Train Acc: 0.061 | Val Acc: 0.072 | Train Loss: 3.5663 | Val Loss: 3.4538
[Run 5] Epoch 03/40 | Train Acc: 0.061 | Val Acc: 0.072 | Train Loss: 3.5663 | Val Loss: 3.4538
[Run 5] Epoch 04/40 | Train Acc: 0.090 | Val Acc: 0.102 | Train Loss: 3.3985 | Val Loss: 3.2162
[Run 5] Epoch 04/40 | Train Acc: 0.090 | Val Acc: 0.102 | Train Loss: 3.3985 | Val Loss: 3.2162
[Run 5] Epoch 05/40 | Train Acc: 0.118 | Val Acc: 0.138 | Train Loss: 3.2274 | Val Loss: 3.2526
[Run 5] Epoch 05/40 | Train Acc: 0.118 | Val Acc: 0.138 | Train Loss: 3.2274 | Val Loss: 3.2526
[Run 5] Epoch 06/40 | Train Acc: 0.142 | Val Acc: 0.160 | Train Loss: 3.0723 | Val Loss: 3.1945
[Run 5] Epoch 06/40 | Train Acc: 0.142 |

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr,███████▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁
train_acc,▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▆▅▆▆▆▆▆▇▇▇▇▇▇▇▇█▇█████
train_loss,█▇▇▇▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_acc,▁▁▁▂▂▃▂▃▄▄▄▄▄▅▅▄▆▅▆▆▇▆▇▇▇▇██▇▇▇▇██████▇█
val_f1_macro,▁▁▁▂▂▂▂▂▃▄▄▄▄▅▅▄▆▅▅▅▆▅▇▇▇▇█▇▇▇▇▇██████▇▇
val_f1_weighted,▁▁▁▂▂▂▂▂▃▄▄▄▄▅▅▄▆▅▅▅▆▅▇▇▇▇█▇▇▇▇▇██████▇▇
val_loss,█▇▇▆▆▆▇▆▄▄▄▄▄▄▃▅▃▃▃▄▂▃▂▂▂▂▂▁▂▂▂▁▂▁▁▁▁▁▁▁
val_precision_macro,▁▁▁▂▃▃▂▂▄▄▄▄▄▅▆▅▇▆▆▅▇▅▇▇▇█▇▇▇▇▇▇██████▇█
val_recall_macro,▁▁▁▂▂▃▂▃▄▄▄▄▄▅▅▄▆▅▆▆▇▆▇▇▇▇██▇▇▇▇██████▇█
best_val_acc,0.5125


# Evaluacion de los modelos